In [1]:
import numpy as np
import pandas as pd
from cvxopt import matrix, solvers
from scipy.optimize import minimize

#Question-1

In [2]:
def func(x):
    return (-x[0] - x[1] - x[2] - x[3])

# Refer scipy documentation for why minus sign in constraints. Check constrainsts section
# LINK : https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize

def constraint1(x):
    return -1*((x[0]-x[1])**2 + (x[2] + 2*x[3])**4 - 5)

def constraint2(x):
    return -1*(x[0]+2*x[1]+3*x[2]+4*x[3]-6)

cons = [{'type': 'ineq', 'fun': constraint1}, {'type': 'ineq', 'fun': constraint2}]
bounds = [(0, None), (0, None), (0, None), (0, None)]
x0 = [0, 0, 0, 0]

result = minimize(func, x0, method='SLSQP', bounds=bounds, constraints=cons)

print(result)
print("Optimal solution:")
print("x1 =", np.round(result.x[0], decimals = 2))
print("x2 =", np.round(result.x[1], decimals = 2))
print("x3 =", np.round(result.x[2], decimals = 2))
print("x4 =", np.round(result.x[3], decimals = 2))

# x1 = 2.8497099366136913e+31
# x2 = 2.8497099366136895e+31
# x3 = 3.5890973769734204e+31
# x4 = 4.328484790104133e+31

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -4.745355992499908
       x: [ 3.491e+00  1.255e+00  0.000e+00  0.000e+00]
     nit: 8
     jac: [-1.000e+00 -1.000e+00 -1.000e+00 -1.000e+00]
    nfev: 42
    njev: 8
Optimal solution:
x1 = 3.49
x2 = 1.25
x3 = 0.0
x4 = 0.0


#Question-2

In [3]:
data_path = '/content/2_col_revised.xlsx'
col_names = ['area', 'price']
data = pd.read_excel(data_path, header=None, names=col_names)
# header = none treat first row as data and does not replace it with col_names
# print(data.iloc[:-1, 0].values)
x = data.iloc[:, 0].values.reshape(-1,1)
y = data.iloc[:, 1].values
# print(y.shape)
n = len(x)
x[n-1]= 5
R = 8
degrees = [1, 2, 3, 4, 5, 6, 7]
r = 15

def func(a):
    return np.sum(a)

def constraint(a, degree):
    fitted_values = np.sum(a * (x**degree)) - y
    return np.sum(fitted_values)

print("Price of 5-thousand square feet house:-")
for degree in degrees:
    initial_guess = np.ones(degree + 1)

    poly_constraint = {'type': 'eq', 'fun': lambda alpha: constraint(alpha, degree)}

    bounds = [(0, None) for _ in range(degree + 1)]
    result = minimize(func, initial_guess, method='SLSQP', constraints=[poly_constraint], bounds=bounds)
    # print(result)

    if result.success and result.fun <= r:
        price = np.polyval(result.x, R)*100
        print(f"For degree: {degree}, Price: {price:.2f}")

FileNotFoundError: ignored

#Question-3

In [ ]:
R = 28

The variable soln will be a dictionary with the following keys:

**'x':** The optimal variable values (the solution to the linear programming problem). These are the values that minimize the linear objective function subject to the given constraints.

**'y':** The Lagrange multipliers associated with the inequality constraints. These represent the dual variables associated with the inequalities.

**'s':** The values of the slack variables associated with the inequality constraints. Slack variables measure the surplus or slack in the constraints.

**'z':** The values of the dual variables associated with the variable bounds.

**'status':** A string indicating the status of the optimization. Common status values include:

**'optimal':** The optimization was successful, and an optimal solution was found.
**'infeasible':** The linear programming problem is infeasible (no feasible solution exists).
**'unbounded':** The linear programming problem is unbounded (there is no optimal solution).
**'primal objective':** The optimal value of the linear objective function at the optimal solution.

**'dual objective': **The optimal value of the dual objective function at the optimal solution.

**'residual as primal infeasibility certificate':** The residual primal infeasibility certificate.

**'residual as dual infeasibility certificate':** The residual dual infeasibility certificate.

**'relative primal infeasibility':** The relative primal infeasibility.

**'relative dual infeasibility':** The relative dual infeasibility.

**'iterations':** The number of iterations performed by the linear programming solver.

In [ ]:
# considering x2 = x3 - x4 s.t x3>=0, x4>=0
# min 3*x1 − 4*x3 + 4*x4

A = np.array([[1, 3, -3], [2, -1, 1], [-1, 4, -4], [-1, 0, 0], [0, -1, 0], [0, 0, -1]])
b = np.array([[12], [20], [-5], [0], [0], [0]])
c = np.array([[3], [-4], [4]])
soln = solvers.lp(matrix(c,tc='d'),matrix(A,tc='d'),matrix(b,tc='d'))
# The tc='d' specifies that the data type of the matrix elements should be 'double' (float).
# print(soln)
sol_p = np.round(soln['primal objective'], decimals=5)
print(np.round_(soln['x'],decimals=5))
print("Minimum value is",sol_p)

In [ ]:
# dual of the problem
# max -12*y1 - 20*y2 + 5*y3

# max -b^T lmb1                                                    min c^T x
  # s.t. A^T lmb1 >= -c => -A^T lmb1 <= c   this is dual for       s.t. Ax <= b => Ax - b <= 0
  #      lmb1 >= 0      => -lmb1 <= 0                              x>= 0 => -x <= 0

A = np.array([[-1, -2, 1], [-3, 1, -4], [3, -1, 4], [-1, 0, 0], [0, -1, 0], [0, 0, -1]])
b = np.array([[3], [-4], [4], [0], [0], [0]])
c = np.array([[12], [20], [-5]])
soln = solvers.lp(matrix(c,tc='d'),matrix(A,tc='d'),matrix(b,tc='d'))
sol_d = np.round(-soln['primal objective'], decimals=5)
print(np.round_(soln['x'],decimals=5))
print("Maximum value is", sol_d)

In [ ]:
#duality gap

duality_gap = (sol_p - sol_d)
print(duality_gap)

#Question-4

In [ ]:
# min (R + 3)x1 + (R + 41)x2
# R = 28

A = np.array([[3, -1], [7, 11], [-1, 0], [0, -1]])
b = np.array([[12], [88], [0], [0]])
c = np.array([[R+3], [R+41]])
soln = solvers.lp(matrix(c,tc='d'),matrix(A,tc='d'),matrix(b,tc='d'))
sol_p = np.round(soln['primal objective'], decimals=5)
print(np.round_(soln['x'],decimals=5))
print("Minimum value is",sol_p)

In [ ]:
# dual of the problem
# max -12*y1 - 88*y2
# min 12*y1 + 88*y2
# lmb1 = [y1,y2]^T

# max -b^T lmb1 => min b^T lmb1                                           min c^T x
  # s.t. A^T lmb1 >= -c => -A^T lmb1 <= c   this is dual for              s.t. Ax <= b => Ax - b <= 0
  #      lmb1 >= 0      => -lmb1 <= 0                                     x>= 0 => -x <= 0

A = np.array([[-3, -7], [1, -11], [-1, 0], [0, -1]])
b = np.array([[R+3], [R+41], [0], [0]])
c = np.array([[12], [88]])
soln = solvers.lp(matrix(c,tc='d'),matrix(A,tc='d'),matrix(b,tc='d'))
sol_d = np.round(-soln['primal objective'], decimals=5)
print(np.round_(soln['x'],decimals=5))
print("Maximum value is", sol_d)

In [ ]:
#duality gap

duality_gap = (sol_p - sol_d)
print(duality_gap)

#Question-5

In [ ]:
# min Rx1 − (R − 1)x2

A = np.array([[3, -2], [3, -2], [-1, 0], [0, -1]])
b = np.array([[1], [6], [0], [0]])
c = np.array([[R], [-R + 1]])
# print(c)
soln = solvers.lp(matrix(c,tc='d'),matrix(A,tc='d'),matrix(b,tc='d'))

In [ ]:
# dual of the problem
# max -1*y1 - 6*y2

A = np.array([[-3, -3], [2, 2], [-1, 0], [0, -1]])
b = np.array([[R], [-R + 1], [0], [0]])
c = np.array([[1], [6]])
soln = solvers.lp(matrix(c,tc='d'),matrix(A,tc='d'),matrix(b,tc='d'))

#Question-6

In [ ]:
# max z = x1 + x2 + x3

A = np.array([[3, 2, 1], [2, 1, 2], [-1, 0, 0], [0, -1, 0], [0, 0, -1]])
b = np.array([[3], [2], [0], [0], [0]])
c = np.array([[-1], [-1], [-1]])
soln = solvers.lp(matrix(c,tc='d'),matrix(A,tc='d'),matrix(b,tc='d'))
sol_p = np.round(-soln['primal objective'], decimals=5)
print(np.round_(soln['x'],decimals=5))
print("Maximum value is",sol_p)

In [ ]:
# dual of the problem
# min 3*y1 + 2*y2

A = np.array([[-3, -2], [-2, -1], [-1, -2], [-1, 0], [0, -1]])
b = np.array([[-1], [-1], [-1], [0], [0]])
c = np.array([[3], [2]])
soln = solvers.lp(matrix(c,tc='d'),matrix(A,tc='d'),matrix(b,tc='d'))
sol_d = np.round(soln['primal objective'], decimals=5)
print(np.round_(soln['x'],decimals=5))
print("Minimum value is", sol_d)

In [ ]:
#duality gap

duality_gap = (sol_p - sol_d)
print(duality_gap)

#Question-7

In [ ]:
# max z = 17x1 + 18x2

A = np.array([[2, 1], [1, 1], [2, 4], [-1, 0], [0, -1]])
b = np.array([[1000], [600], [2000], [0], [0]])
c = np.array([[-(R+2)], [-(R+3)]])
soln = solvers.lp(matrix(c,tc='d'),matrix(A,tc='d'),matrix(b,tc='d'))
sol_p = np.round(-soln['primal objective'], decimals=5)
print(np.round_(soln['x'],decimals=5))
print("Maximum value is",sol_p)

In [ ]:
# dual of the problem
# min 1000*y1 + 600*y2 + 2000*y3

A = np.array([[-2, -1, -2], [-1, -1, -4], [-1, 0, 0], [0, -1, 0], [0, 0, -1]])
b = np.array([[-(R+2)], [-(R+3)], [0], [0], [0]])
c = np.array([[1000], [600], [2000]])
soln = solvers.lp(matrix(c,tc='d'),matrix(A,tc='d'),matrix(b,tc='d'))
sol_d = np.round(soln['primal objective'], decimals=5)
print(np.round_(soln['x'],decimals=5))
print("Minimum value is", sol_d)

In [ ]:
#duality gap

duality_gap = np.round((sol_p - sol_d), decimals = 4)
print(np.round(duality_gap,decimals=2))

#Question-8

In [ ]:
# min 2x1 + 3x2 + 10x3

A = np.array([[1, 0, 2], [-1, 0, -2], [0, -1, -1], [0, 1, 1], [-1, 0, 0], [0, -1, 0], [0, 0, -1]])
b = np.array([[0], [0], [-1], [1], [0], [0], [0]])
c = np.array([[2], [3], [10]])
soln = solvers.lp(matrix(c,tc='d'),matrix(A,tc='d'),matrix(b,tc='d'))
sol_p = np.round(soln['primal objective'], decimals=5)
print(np.round_(soln['x'],decimals=5))
print("Minimum value is",sol_p)

In [ ]:
# dual of the problem
# max y3 - y4

A = np.array([[-1, 1, 0, 0], [0, 0, 1, -1], [-2, 2, 1, -1], [-1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, -1]])
b = np.array([[2], [3], [10], [0], [0], [0], [0]])
c = np.array([[0], [0], [-1], [1]])
soln = solvers.lp(matrix(c,tc='d'),matrix(A,tc='d'),matrix(b,tc='d'))
sol_d = np.round(-soln['primal objective'], decimals=5)
print(np.round_(soln['x'],decimals=5))
print("Maximum value is", sol_d)

In [ ]:
#duality gap

duality_gap = np.round((sol_p - sol_d), decimals = 5)
print(duality_gap)

#Question-9

In [ ]:
# considering x2 = x3 - x4 s.t x3>=0, x4>=0
# min 3*x1 − 4*x3 + 4*x4

A = np.array([[1, 3, -3], [2, -1, 1], [-1, 4, -4], [-1, 0, 0], [0, -1, 0], [0, 0, -1]])
b = np.array([[12], [20], [-5], [0], [0], [0]])
c = np.array([[3], [-4], [4]])
soln = solvers.lp(matrix(c,tc='d'),matrix(A,tc='d'),matrix(b,tc='d'))
sol_p = np.round(soln['primal objective'], decimals=5)
print(np.round_(soln['x'],decimals=5))
print("Minimum value is",sol_p)

In [ ]:
# dual of the problem
# max -12*y1 - 20*y2 + 5*y3

A = np.array([[-1, -2, 1], [-3, 1, -4], [3, -1, 4], [-1, 0, 0], [0, -1, 0], [0, 0, -1]])
b = np.array([[3], [-4], [4], [0], [0], [0]])
c = np.array([[12], [20], [-5]])
soln = solvers.lp(matrix(c,tc='d'),matrix(A,tc='d'),matrix(b,tc='d'))
sol_d = np.round(-soln['primal objective'], decimals=5)
print(np.round_(soln['x'],decimals=5))
print("Maximum value is", sol_d)

In [ ]:
#duality gap

duality_gap = np.round((sol_p - sol_d), decimals = 4)
print(duality_gap)

#Question-10

In [ ]:
# max 15x1 + 18x2

A = np.array([[-3, -1], [-1, -4], [1, 1], [-1, 0], [0, -1]])
b = np.array([[-3], [-4], [5], [0], [0]])
c = np.array([[-R], [-(R+3)]])
soln = solvers.lp(matrix(c,tc='d'),matrix(A,tc='d'),matrix(b,tc='d'))
sol_p = np.round(-soln['primal objective'], decimals=5)
print(np.round_(soln['x'],decimals=5))
print("Maximum value is",sol_p)

In [ ]:
# dual of the problem
# min -3*y1 - 4*y2 + 5*y3

A = np.array([[3, 1, -1], [1, 4, -1], [-1, 0, 0], [0, -1, 0], [0, 0, -1]])
b = np.array([[-R], [-(R+3)], [0], [0], [0]])
c = np.array([[-3], [-4], [5]])
soln = solvers.lp(matrix(c,tc='d'),matrix(A,tc='d'),matrix(b,tc='d'))
sol_d = np.round(soln['primal objective'], decimals=5)
print(np.round_(soln['x'],decimals=5))
print("Minimum value is", sol_d)

In [ ]:
#duality gap

duality_gap = np.round((sol_p - sol_d), decimals = 5)
print(duality_gap)

#Question-11

In [ ]:
# min 15x1 + 31x2

A = np.array([[2, 1], [-3, -4], [-2, 3], [-1, 0], [0, -1]])
b = np.array([[4], [-24], [-6], [0], [0]])
c = np.array([[R], [2*R+1]])
soln = solvers.lp(matrix(c,tc='d'),matrix(A,tc='d'),matrix(b,tc='d'))

In [ ]:
# dual of the problem
# max -4*y1 + 24*y2 + 6*y3

A = np.array([[-2, 3, 2], [-1, 4, -3], [-1, 0, 0], [0, -1, 0], [0, 0, -1]])
b = np.array([[R], [2*R+1], [0], [0], [0]])
c = np.array([[4], [-24], [-6]])
soln = solvers.lp(matrix(c,tc='d'),matrix(A,tc='d'),matrix(b,tc='d'))

#Question-12

In [ ]:
# min 18x1 + x2

A = np.array([[1, 2], [-4, -3], [3, 1], [-3, -1], [-1, 0], [0, -1]])
b = np.array([[3], [-6], [3], [-3], [0], [0]])
c = np.array([[R+3], [1]])
soln = solvers.lp(matrix(c,tc='d'),matrix(A,tc='d'),matrix(b,tc='d'))
sol_p = np.round(soln['primal objective'], decimals=5)
print(np.round_(soln['x'],decimals=5))
print("Minimum value is",sol_p)

In [ ]:
# dual of the problem
# max -3*y1 + 6*y2 + 3*y3 - 3*y4

A = np.array([[-1, 4, 3, -3], [-2, 3, 1, -1], [-1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, -1]])
b = np.array([[R+3], [1], [0], [0], [0], [0]])
c = np.array([[3], [-6], [-3], [3]])
soln = solvers.lp(matrix(c,tc='d'),matrix(A,tc='d'),matrix(b,tc='d'))
sol_d = np.round(-soln['primal objective'], decimals=5)
print(np.round_(soln['x'],decimals=5))
print("Maximum value is", sol_d)

In [ ]:
#duality gap

duality_gap = np.round((sol_p - sol_d), decimals = 5)
print(duality_gap)

#Question-13

In [ ]:
# min -3x1 + x2

A = np.array([[1, 2], [-1, -2], [2, -2], [-2, 2], [-1, 0], [0, -1]])
b = np.array([[0], [0], [9], [-9], [0], [0]])
c = np.array([[-3], [1]])
soln = solvers.lp(matrix(c,tc='d'),matrix(A,tc='d'),matrix(b,tc='d'))

In [ ]:
# dual of the problem
# max 9*y3 - 9*y4

A = np.array([[1, -1, 2, -2], [2, -2, -2, 2], [-1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, -1]])
b = np.array([[-3], [1], [0], [0], [0], [0]])
c = np.array([[0], [0], [-9], [9]])
soln = solvers.lp(matrix(c,tc='d'),matrix(A,tc='d'),matrix(b,tc='d'))
sol_d = np.round(-soln['primal objective'], decimals=5)